# Universidad Nacional Autónoma de México
# Instituto de Investigaciones en Matemáticas Aplicadas y Sistemas
# Bases de Datos No Estructurados 
# Práctica 2: Cassandra

# Estudiantes:

# -López Sánchez Misael

# -Tapia López José de Jesús

## Objetivo
* <p style='text-align: justify;'>Desarrollar un sistema de consulta de libros en la que multiples usuarios puedan introducir puntuaciones respecto a que tanto les ha agradado un libro en general y poder realizar consultas sobre gustos literarios.</p>

* <p style='text-align: justify;'>Desarrollar una plataforma de libros.</p>
* <p style='text-align: justify;'>Debe incluir producto, cliente y califiación.</p>
* <p style='text-align: justify;'>Cada cliente debe tener información asociada (nombre, pais, membresía, etc).</p>
* <p style='text-align: justify;'>Además, cada cliente debe poder elegir una categoría para cada uno de sus libros (fantasía, misterio, etc).</p>
* <p style='text-align: justify;'> Todos deben participar como clientes</p>


## Introducción.
### ¿Qué es Apache Cassandra?
<p style='text-align: justify;'>Cassandra es un sistema de código abierto distribuido de gestión de base de datos diseñado para manejar grandes cantidades de datos a través de muchos servidores de conveniencia, proporcionando una alta disponibilidad sin ningún punto único de fallo, ofrece un nivel muy elevado de compatibilidad para los clústers que abarcan múltiples centros de datos, con la replicación asíncrona sin muestreo, lo que permite operaciones de baja latencia para todos los clientes es una base de datos tipo $\textit{NoSQL}$ distribuida y basada en un modelo de almacenamiento de 'clave'-'valor' escrita en Java.</p>

<p style='text-align: justify;'>El modelo de datos de Cassandra consiste en partiocionar las filas, que son reorganizadas en tablas. Las clves primarias de cada tabla tiene un primer componente; que es la $\textit{partition key}$, y un un segundo componete; $\textit{que es el cluster key}$. 
    
* <p style='text-align: justify;'>La $\textit{partition key}$ es la $\textit{primary key}$ en caso de ser simple. Toda la información que se agrupe bajo un mismo renglón será guardada en un mismo nodo del clúster.</p>
* <p style='text-align: justify;'> La $\textit{cluster key}$ existe cuando la $\textit{primary key}$ está compuesta, y esta es desde la segunda parte en adelante, la cual se tomará para formar las familias de columnas, cada valor se agrupará bajo esos grupos, los cuales se mantendrán ordenados en el almacenamiento.</p>

Dentro de una partición las filas son agrupadas por columnas restantes de la clave. 
    
Cassandra no soporta $\textit{Joins}$ o $\textit{subconsultas}$ , sino que se enfatiza en la desnormalización a través de características como colecciones.</p>

<img src="capCassandra.png" heigth="400" width="400">

### Características de Cassandra.
 Apache Cassandra ofrece una serie de beneficios que optimizan el potencial de las aplicaciones online como:
 
* Arquitectura escalable.
* Diseño activo de inicio a fin pues los nodos se pueden escribir y leer.
* Rendimiento a escala líneal pues se puede añadir nuevos nodos sin tener que frenar el ritmo.
* Disponibilidad continua.
* Detección de fallos y recuperación transparente pues los nodos pueden ser fácilmente restaurados.
* Modelo flexible y dinámico.
* protección de datos solida.
* CQL (Lenguaje de Consulta Cassandra) el cual es similar a SQL que consigue que la transición de una base de datos relacional sea muy sencilla.

<img src="Apcassandra.png" heigth = "300" width="400">

## Desarrollo

### Instalamos
<p style='text-align: justify;'>Para comenzar primero es necesario instalar previamente Apache Cassandra en nuestro sistema operativo, para esto descargamos casandra desde el portal www-us.apache.org. Recordemos que primero debemos tener ciertos prerrequisitos como tener instalado el jdk-8** de Java, pues Cassandra esta desarrollado en este lenguaje; y posteriormente instalar el driver que funcionará de conexión entre $\textbf{Cassandra}$ y $\textbf{Python}$ con el siguiente comando: </p>

* pip install cassandra-driver==3.22 --force-reinstall --upgrade

In [1]:
#pip install cassandra-driver


### Verificamos la instalación 
Checamos que la instalación se haya completado y revisamos la versión que tenemos de Cassandra que en este caso es la 3.22.0

In [2]:
import cassandra
print(cassandra.__version__)


3.22.0


### Conexión a Cassandra

<p style='text-align: justify;'>Antes de que podamos comenzar a ejecutar consultas en un clúster Cassandra, necesitamos configurar una instancia de Cluster. Como su nombre lo indica, normalmente tendrá una instancia de Cluster para cada clúster de Cassandra con el que desee interactuar.</p>

La forma más sencilla de crear un clúster es así:


In [1]:
#from cassandra.cluster import Cluster
#cluster = Cluster()


Esto intentará conectarse a una instancia de Cassandra en su máquina local (127.0.0.1). También puede especificar una lista de direcciones IP para los nodos en su clúster:

In [3]:
#cluster = Cluster(['192.168.0.1', '192.168.0.2'])


<p style='text-align: justify;'>El conjunto de direcciones IP que pasamos al Cluster es simplemente un conjunto inicial de puntos de contacto. Después de que el controlador se conecte a uno de estos nodos, descubrirá automáticamente el resto de los nodos en el clúster y se conectará a ellos, por lo que no necesita enumerar cada nodo en su clúster.</p>

<p style='text-align: justify;'>Si necesita usar un puerto no estándar, usar SSL o personalizar el comportamiento del controlador de alguna otra manera, esta es la forma para hacerlo:</p>

In [4]:
#cluster = Cluster(['192.168.0.1', '192.168.0.2'], port=..., 
                    ssl_context=...)


<p style='text-align: justify;'>Instanciar un clúster en realidad no nos conecta a ningún nodo. Para establecer conexiones y comenzar a ejecutar consultas, necesitamos una sesión, que se crea llamando a $\textbf{cluster.connect()}$. El método $\textbf{connect()}$ toma una $\textit{keyspace}$ como argumento opcional que establece la $\textit{keyspace}$ predeterminado para todas las consultas realizadas a través de esa sesión. Sin embargo, en nuestro caso no le colocamos un argumento pues posteriormente crearemos la keyspace que usaremos para nuestra tabla de la práctica </p>

In [2]:
from cassandra.cluster import Cluster

#Realizamos la conexión entre Python y el Clúster de Cassandra
cluster= Cluster()
session = cluster.connect()


<p style='text-align: justify;'> Creamos una $\textit{keyspace}$ llamada uprofile donde vamos a crear nuestra tabla que usaremos para esta práctica: </p>

In [2]:
#session.execute('CREATE KEYSPACE IF NOT EXISTS uprofile WITH replication = 
                {\'class\': \'NetworkTopologyStrategy\', \'datacenter1\' : \'1\' }')

### Diseño de la base de datos.
Creamos el diseño de la base de datos de nuestros libros donde los campos a utilizar serán:
* membresia - Integer
* nombre_cliente - Text
* pais - Text
* nombre_libro - Text
* categoria - Text
* puntuacion - Float

<p style='text-align: justify;'>Por llave primaria tomamos los campos $\textbf{categoria}$ y $\textbf{nombre_libro, membresia}$ como $\textit{partition key}$ y $\textit{cluster key}$ respectivamente; pues al momento de realizar consultas lo principal que nos gustaria conocer son los libros que se estan consultando y el individuo de la consulta.</p>

<img src="scalaCas.png" width="530" height="200">

Ahora bien, para empezar con el desarrollo de la práctica empezamos con la creación de la tabla:

In [7]:
#session.execute('DROP TABLE uprofile.plataforma_de_libros');

In [9]:
# Creamos la tabla

session.execute('CREATE TABLE uprofile.plataforma_de_libros (membresia int, nombre_cliente text,pais text, nombre_libro text, categoria text, puntuacion float, PRIMARY KEY (categoria,nombre_libro,membresia))');

<p style='text-align: justify;'>Realizamos más de 50 registros a la base de datos donde se cuentan con datos de diferentes usuarios de diferentes paises que han leido libros en común y cada uno de ellos les ha dado una calificación diferente, por lo que buscaremos después buscar a aquellos usuarios que tienen gustos literarios comunes.</p>

In [10]:
insertar_datos = session.prepare("INSERT INTO  uprofile.plataforma_de_libros (membresia, nombre_cliente, pais, nombre_libro, categoria, puntuacion) VALUES (?,?,?,?,?,?)")


In [11]:
# Le agregamos unos registros...

insertar_datos = session.prepare("INSERT INTO  uprofile.plataforma_de_libros (membresia, nombre_cliente, pais, nombre_libro, categoria, puntuacion) VALUES (?,?,?,?,?,?)")
session.execute(insertar_datos, [418004719,'José de Jesús Tapia López','México','Rayuela','Antinovela',9])
session.execute(insertar_datos, [673511509,'Misael López Sánchez','España','Drácula','Terror',8])
session.execute(insertar_datos, [123456789,'Alejandro Pimentel Alarcón','Francia','Madame Bovary','Romance',8])

In [12]:
session.execute(insertar_datos, [312284510,'Armando Aguilar Léon','México','Rayuela','Antinovela',8])
session.execute(insertar_datos, [315332160,'Alfredo Aguilar Valdez','USA','La Biblia','Religión',7])
session.execute(insertar_datos, [315023046,'Jonhattan Alanis Blancarte','España','Gilgamesh','Epopeya',6])
session.execute(insertar_datos, [314026161,'Fernanda Aquino Cruz','Colombia','Divina comedia','Epopeya',8])
session.execute(insertar_datos, [315001091,'Epifania Avendaño Montesinos','México','Orgullo y Prejuicio','Drama ',10])
session.execute(insertar_datos, [315232905,'Yazmín Baltazar Saucedo','Francia','Decamerón','Cuentos',10])
session.execute(insertar_datos, [315138670,'María Bolon Sánchez','China','El extranjero','Policial',9])
session.execute(insertar_datos, [314129420,'Patricia Caire Ascencio','Rusia','Don Quijote de la Mancha','Aventura',8])
session.execute(insertar_datos, [316294364,'Valeria Camacho Ávila','Rusia','Harry Potter','Ficción',10])
session.execute(insertar_datos, [315042416,'Joanna Carmona Lara','India','El señor de los anillos','Ficción',10])
session.execute(insertar_datos, [315200919,'Omar Castañeda Díaz','Argentina','La Iliada','Epopeya',5])
session.execute(insertar_datos, [315132032,'César Collado Morales','Rusia','La Odisea','Epopeya',1])
session.execute(insertar_datos, [315132159,'Raúl Corona Flores','México','Lolita','Tragicomedia',4])
session.execute(insertar_datos, [416058606,'Arath de la Parra Arias','Colombia','Pedro Páramo','Realismo',8])
session.execute(insertar_datos, [404114851,'Minerva Del Real Martínez','España','Harry Potter','Ficción',7])
session.execute(insertar_datos, [316281810,'Angel Dimas Huesca','España','Las mil y una noches','Cuentos',6])
session.execute(insertar_datos, [316032454,'Alejandro Flores Delgado','China','Los viajes de Gulliver','Aventura',6])
session.execute(insertar_datos, [315083532,'Leonardo Flores Ochoa','Argentina','Edipo Rey','Tragedia',6])
session.execute(insertar_datos, [418003712,'Adrián Flores Pulido','USA','Fausto','Tragedia',10])
session.execute(insertar_datos, [315335130,'Marcela Fuentes Cerda','USA','Fausto','Tragedia',7])
session.execute(insertar_datos, [315137817,'Alberto Gómez Correa','Canadá','El extranjero','Policial',8])
session.execute(insertar_datos, [315212992,'Esmeralda Gónzalez Palacios','Canadá','Harry Potter','Ficcion',8])
session.execute(insertar_datos, [417042329,'Jasson Gónzalez Ramos','Japón','Edipo Rey','Tragedia',9])
session.execute(insertar_datos, [316152976,'Andrea Gónzalez Rodríguez','Japón','Gilgamesh','Epopeya',7])
session.execute(insertar_datos, [113001587,'Yvette González Chávez','Canadá','Gilgamesh','Epopeya',7])
session.execute(insertar_datos, [315309490,'Brandon Hernandez Nava','China','Lolita','Tragicomedia',4])
session.execute(insertar_datos, [419004066,'Ibeth Hernández Corona','Uruguay','Los viajes de Gulliver','Aventura',5])
session.execute(insertar_datos, [315045606,'Catherine Jiménez Jiménez','Uruguay','La Odisea','Epopeya ',8])
session.execute(insertar_datos, [111002674,'Mauricio José Barreiro','India','La Iliada','Epopeya ',6])
session.execute(insertar_datos, [315284557,'Yoselyn Juárez Casimiro','Italia','Los viajes de Gulliver','Aventura',7])
session.execute(insertar_datos, [316178352,'Sebastián_Lazcano Espindola','India','Edipo Rey','Tragedia',5])
session.execute(insertar_datos, [315293230,'Adriana Leal Molina','Italia','La Biblia','Religión',8])
session.execute(insertar_datos, [417039617,'César López Domínguez','Alemania','Don Quijote de la Mancha','Aventura',7])
session.execute(insertar_datos, [315292178,'Eduardo López López','Alemania','Gilgamesh','Epopeya',4])
session.execute(insertar_datos, [315018086,'Diego López Reyes','Argentina','La Biblia','Religión',7])
session.execute(insertar_datos, [315295306,'Antonio Mani Yañez','Guatemala','Divina comedia','Epopeya ',8])
session.execute(insertar_datos, [316117199,'Iván Medina Martínez','Guatemala','Orgullo y Prejuicio','Drama',6])
session.execute(insertar_datos, [315026487,'Alexis Medina Mauricio','Guatemala','El señor de los anillos','Ficción',10])
session.execute(insertar_datos, [315316409,'Augusto Medina Ramírez','Panama','Divina comedia','Epopeya',6])
session.execute(insertar_datos, [314319373,'Nadine Mejía Cerda','Panama','Orgullo y Prejuicio','Drama',10])
session.execute(insertar_datos, [314105086,'Jazmín Mendoza Vara','Ecuador','Decamerón','Drama',8])
session.execute(insertar_datos, [418003781,'Evelyn Salinas Ramírez','Ecuador','Don Quijote de la Mancha','Aventura',10])
session.execute(insertar_datos, [316063443,'Susan Morales Gónzalez','España','El extranjero','Policial',10])

In [13]:
session.execute(insertar_datos, [418004719,'José de Jesús Tapia López','México','Desde mi cielo','Drama',9])
session.execute(insertar_datos, [312284510,'Armando Aguilar Léon','México','Romeo y Julieta','Drama',10])
session.execute(insertar_datos, [418004719,'José de Jesús Tapia López','México','La Biblia','Religión',6])
session.execute(insertar_datos, [418004719,'José de Jesús Tapia López','México','Romeo y Julieta','Drama',10])
session.execute(insertar_datos, [123456789,'Alejandro Pimentel Alarcón','Francia','Ana Karenina','Romance',8])
session.execute(insertar_datos, [123456789,'Alejandro Pimentel Alarcón','Francia','Desde mi cielo','Drama ',10])
session.execute(insertar_datos, [315232905,'Yazmín Baltazar Saucedo','Francia','Desde mi cielo','Drama',9])
session.execute(insertar_datos, [315138670,'María Bolon Sánchez','China','Desde mi cielo','Drama',9])
session.execute(insertar_datos, [123456789,'Alejandro Pimentel Alarcón','Francia','El gato negro','Terror',7])
session.execute(insertar_datos, [123456789,'Alejandro Pimentel Alarcón','Francia','Cuentos para pensar','Cuentos',10])
session.execute(insertar_datos, [418004719,'José de Jesús Tapia López','México','Carrie','Ficción',6])
session.execute(insertar_datos, [418004719,'José de Jesús Tapia López','México','Ana Karenina','Romance',8])
session.execute(insertar_datos, [315132032,'César Collado Morales','Rusia','Ana Karenina','Romance',10])
session.execute(insertar_datos, [315132032,'César Collado Morales','Rusia','Misery','Terror',7])
session.execute(insertar_datos, [416058606,'Arath de la Parra Arias','Colombia','El gato negro','Terror',7])
session.execute(insertar_datos, [404114851,'Minerva Del Real Martínez','España','Cuentos para pensar','Ficción',10])
session.execute(insertar_datos, [404114851,'Minerva Del Real Martínez','España','Las aventuras de Tom Sawyer','Aventura',7])
session.execute(insertar_datos, [404114851,'Minerva Del Real Martínez','España','Los ojos de mi princesa','Romance',6])
session.execute(insertar_datos, [315083532,'Leonardo Flores Ochoa','Argentina','Las aventuras de Tom Sawyer','Aventura',6])
session.execute(insertar_datos, [315335130,'Marcela Fuentes Cerda','USA','Las aventuras de Tom Sawyer','Aventura',6])
session.execute(insertar_datos, [315335130,'Marcela Fuentes Cerda','USA','Viaje al centro de la tierra','Aventura',7])
session.execute(insertar_datos, [315137817,'Alberto Gómez Correa','Canadá','Viaje al centro de la tierra','Aventura',7])
session.execute(insertar_datos, [315212992,'Esmeralda Gónzalez Palacios','Canadá','Solaris','Ficcion',8])
session.execute(insertar_datos, [315212992,'Esmeralda Gónzalez Palacios','Canadá','Solaris','Ficcion',7])
session.execute(insertar_datos, [315212992,'Esmeralda Gónzalez Palacios','Canadá','Los ojos de mi princesa','Romance',7])
session.execute(insertar_datos, [673511509,'Misael López Sánchez','España','Bajo la misma estrella','Drama',7])
session.execute(insertar_datos, [315309490,'Brandon Hernandez Nava','China','La isla del tesoro','Aventura',7])
session.execute(insertar_datos, [673511509,'Misael López Sánchez','España','La isla del tesoro','Aventura',8])
session.execute(insertar_datos, [315045606,'Catherine Jiménez Jiménez','Uruguay','Dune','Ficcion',8])
session.execute(insertar_datos, [111002674,'Mauricio José Barreiro','India','El juego de Ender','Ficción ',6])
session.execute(insertar_datos, [315284557,'Yoselyn Juárez Casimiro','Italia','El juego de Ender','Ficción ',6])
session.execute(insertar_datos, [673511509,'Misael López Sánchez','España','Neuromante','Ficción ',8])
session.execute(insertar_datos, [315293230,'Adriana Leal Molina','Italia','Ana Karenina','Romance',9])
session.execute(insertar_datos, [315293230,'Adriana Leal Molina','Italia','Don Quijote de la Mancha','Aventura',7])
session.execute(insertar_datos, [315293230,'Adriana Leal Molina','Italia','Bajo la misma estrella','Drama',6])
session.execute(insertar_datos, [315018086,'Diego López Reyes','Argentina','El código Da Vinci','Misterio',8])
session.execute(insertar_datos, [315295306,'Antonio Mani Yañez','Guatemala','La piedra lunar','Misterio',8])
session.execute(insertar_datos, [315026487,'Alexis Medina Mauricio','Guatemala','El código Da Vinci','Misterio',8])
session.execute(insertar_datos, [315026487,'Alexis Medina Mauricio','Guatemala','El código Da Vinci','Misterio',9])
session.execute(insertar_datos, [315316409,'Augusto Medina Ramírez','Panama','La sombra del viento','Misterio',8])
session.execute(insertar_datos, [315316409,'Augusto Medina Ramírez','Panama','La sombra del viento','Misterio',8])
session.execute(insertar_datos, [316063443,'Susan Morales Gónzalez','España','Bajo la misma estrella','Drama',7])
session.execute(insertar_datos, [316063443,'Susan Morales Gónzalez','España','Reina Roja','Misterio',8])
session.execute(insertar_datos, [673511509,'Misael López Sánchez','España','Reina Roja','Misterio',7])

In [3]:
import pandas as pd
from IPython.display import display
print('-'*5, "Obtener todos los registros de la tabla", '-'*5, '\n')
consulta = session.execute('select * from uprofile.plataforma_de_libros;')
membresias = []
categorias = []
clientes = []
paises = []
libros = []
puntuaciones = []

for cons in consulta:
    membresias.append(cons.membresia)
    categorias.append(cons.categoria)
    clientes.append(cons.nombre_cliente)
    paises.append(cons.pais)
    libros.append(cons.nombre_libro)
    puntuaciones.append(cons.puntuacion)

#Creamos el data frame
cons2 = pd.DataFrame({'membresia':membresias,'nombre_cliente':clientes,'pais':paises,'nombre_libro':libros,'categoria':categorias,'puntuacion':puntuaciones},
                    columns=['membresia','nombre_cliente','pais','nombre_libro','categoria','puntuacion'])

pd.set_option('display.max_rows', len(cons2))
display(cons2)
pd.reset_option('display.max_rows')

----- Obtener todos los registros de la tabla ----- 



membresia                nombre_cliente       pais  \
0   111002674        Mauricio José Barreiro      India   
1   315284557       Yoselyn Juárez Casimiro     Italia   
2   673511509          Misael López Sánchez     España   
3   123456789    Alejandro Pimentel Alarcón    Francia   
4   315232905       Yazmín Baltazar Saucedo    Francia   
5   316281810            Angel Dimas Huesca     España   
6   314129420       Patricia Caire Ascencio      Rusia   
7   315293230           Adriana Leal Molina     Italia   
8   417039617         César López Domínguez   Alemania   
9   418003781        Evelyn Salinas Ramírez    Ecuador   
10  315309490        Brandon Hernandez Nava      China   
11  673511509          Misael López Sánchez     España   
12  315083532         Leonardo Flores Ochoa  Argentina   
13  315335130         Marcela Fuentes Cerda        USA   
14  404114851     Minerva Del Real Martínez     España   
15  315284557       Yoselyn Juárez Casimiro     Italia   
16  316032454      Alejandro Flores Delgado      China   
17  419004066        Ibeth Hernández Corona    Uruguay   
18  315137817          Alberto Gómez Correa     Canadá   
19  315335130         Marcela Fuentes Cerda        USA   
20  312284510          Armando Aguilar Léon     México   
21  418004719     José de Jesús Tapia López     México   
22  315137817          Alberto Gómez Correa     Canadá   
23  315138670           María Bolon Sánchez      China   
24  316063443        Susan Morales Gónzalez     España   
25  315083532         Leonardo Flores Ochoa  Argentina   
26  316178352   Sebastián_Lazcano Espindola      India   
27  417042329         Jasson Gónzalez Ramos      Japón   
28  315335130         Marcela Fuentes Cerda        USA   
29  418003712          Adrián Flores Pulido        USA   
30  315018086             Diego López Reyes  Argentina   
31  315026487        Alexis Medina Mauricio  Guatemala   
32  315295306            Antonio Mani Yañez  Guatemala   
33  315316409        Augusto Medina Ramírez     Panama   
34  316063443        Susan Morales Gónzalez     España   
35  673511509          Misael López Sánchez     España   
36  315132159            Raúl Corona Flores     México   
37  315309490        Brandon Hernandez Nava      China   
38  418004719     José de Jesús Tapia López     México   
39  404114851     Minerva Del Real Martínez     España   
40  315026487        Alexis Medina Mauricio  Guatemala   
41  315042416           Joanna Carmona Lara      India   
42  316294364         Valeria Camacho Ávila      Rusia   
43  404114851     Minerva Del Real Martínez     España   
44  123456789    Alejandro Pimentel Alarcón    Francia   
45  315001091  Epifania Avendaño Montesinos     México   
46  315293230           Adriana Leal Molina     Italia   
47  316063443        Susan Morales Gónzalez     España   
48  673511509          Misael López Sánchez     España   
49  314105086           Jazmín Mendoza Vara    Ecuador   
50  315138670           María Bolon Sánchez      China   
51  315232905       Yazmín Baltazar Saucedo    Francia   
52  418004719     José de Jesús Tapia López     México   
53  314319373            Nadine Mejía Cerda     Panama   
54  316117199          Iván Medina Martínez  Guatemala   
55  312284510          Armando Aguilar Léon     México   
56  418004719     José de Jesús Tapia López     México   
57  315045606     Catherine Jiménez Jiménez    Uruguay   
58  315212992   Esmeralda Gónzalez Palacios     Canadá   
59  315212992   Esmeralda Gónzalez Palacios     Canadá   
60  123456789    Alejandro Pimentel Alarcón    Francia   
61  315132032         César Collado Morales      Rusia   
62  315293230           Adriana Leal Molina     Italia   
63  418004719     José de Jesús Tapia López     México   
64  315212992   Esmeralda Gónzalez Palacios     Canadá   
65  404114851     Minerva Del Real Martínez     España   
66  123456789    Alejandro Pimentel Alarcón    Francia   
67  315295306            Antonio Mani Yañez  Guatemala   
68

## Aplicación

<p style='text-align: justify;'> La aplicación funciona como un sitema clasificador de libros, donde cada usuario que ha leido un determinado libro puede darle una clasificación en una escala nominal del 1 al 10 de qué tanto le ha gustado el libro.</p>

Al ingresar a la aplicación del programa, al usuario se le presenta un menú con las siguientes opciones: 
* Ingresar un nuevo registro
* Obtener todos los registros de la tabla
* Obtener todos los registros de un cliente
* Obtener la cantidad de libros leídos de un cliente
* Obtener la cantidad de libros leídos de un cliente por categoría
* Obtener la categoría preferida de un usuario
* Obtener los clientes que más disfrutaron de un libro
* Obtener los mejores libros de una categoría dada
* Obtener los libros favoritos de un usuario
* Estadisticas generales
* Salir

<p style='text-align: justify;'>Lo que nos interesa de esta parte es, al momento de realizar consultas sobre la base de datos, descubrir aquellos usuarios que han leido libros en común y ver que calificación le ha dado cada uno, así para que cada usuario nuevo al leer un libro pueda consultar la información que otros usuarios han dejado de ese libro y ver si el libro es bueno o malo de acuerdo a sus gustos clasificados por categoria.</p>

<p style='text-align: justify;'>En la siguiente parte del código hemos puesto algunas líneas de código, que anteriormente lo habíamos usado, para facilitarnos la visualización de la aplicación.</p>

In [4]:
from cassandra.cluster import Cluster
import os
import pandas as pd
from IPython.display import display

#Realizamos la conexión entre Python y el Clúster de Cassandra
#cluster = Cluster(['localhost'],port=6379)
cluster= Cluster()
session = cluster.connect()

insertar_datos = session.prepare("INSERT INTO  uprofile.plataforma_de_libros (membresia, nombre_cliente, pais, nombre_libro, categoria, puntuacion) VALUES (?,?,?,?,?,?)")

#funciones auxiliares para ingresar datos por consola en Python.
os.system('cls')
clear = lambda:os.system('cls')

#Obs: La aplicación no acentos comas así que al registrarlas en Redis se deforman.

def menu_aplication():
    """
    @Author: Jose de Jesus Tapia Lopez, Misael Lopez Sanchez
    summary: Menú que imprime la interfaz gráfica para ser puente entre Python y Cassandra
    Parameters: NULL
    Returns: Pantalla de la intefaz gŕafica
    """
    print('+'*10,"PLATAFORMA DE LIBROS", '+'*10, '\n')
    print('+'*20," MENU ", '+'*20, '\n')
    print("1.- Ingresar nuevo registro")
    print("2.- Obtener todos los registros de la tabla")
    print("3.- Obtener todos los registros de un cliente")
    print("4.- Obtener la cantidad de libros leídos de un cliente")
    print("5.- Obtener la cantidad de libros leídos de un cliente por categoría")
    print("6.- Obtener la categoría preferida de un usuario")
    print("7.- Obtener los clientes que más disfrutaron de un libro")
    print("8.- Obtener los mejores libros de una categoría dada")
    print("9.- Obtener los libros favoritos de un usuario")
    print("10.- Estadísticas generales")
    print("11.- Salir")
    print("")
    
    
def menu_estadisticas():
    """
    @Author: Jose de Jesus Tapia Lopez, Misael Lopez Sanchez
    summary: Menu auxiliar para que el usuario pueda consultar algunas otras estadisticas generales extras
    Parameters: NULL
    Returns: Pantalla de la intefaz gŕafica 
    
    """
    print()
    print('1.- Registros totales por país')
    print('2.- Libros más leidos ')
    print('3.- Puntuaciones de usuarios sobre libros en especifico ')
    print('4.- Regresar')
    print()
    
def estadisticas():
    """
    @Author: Jose de Jesus Tapia Lopez, Misael Lopez Sanchez
    summary: Funcion que realiza operaciones básicas a insertar en la aplicacion general opción 6
    Parameters: Valor de la consulta a generar
    Returns: valor de la consulta especificada
    
    """
    while True:
        print('-'*5, "Estadísticas generales", '-'*5, '\n')
        menu_estadisticas()
        opEst = input('Ingrese una opción >> ')
        print()

        if opEst == '1': # Devuelve el número total de registros que se han realizado por pais
            consultaPais  = session.execute('select pais,membresia,nombre_cliente from uprofile.plataforma_de_libros;')
            #Guardamos valores en DataFrame para poder trabajar consultas desde pandas.
            pais = []
            membresias = []
            nombres = []
            for cons in consultaPais:
                pais.append(cons.pais)
                membresias.append(cons.membresia)
                nombres.append(cons.nombre_cliente)
            #creamos el dataframe
            dfPais = pd.DataFrame({'usuarios':pais,'membresia':membresias,'nombre_cliente':nombres},columns=['usuarios','membresia','nombre_cliente'])
            mipais = input('Introduce el pais: ')
            dl = dfPais[dfPais['usuarios'] == mipais ] #filtramos valores por país
            n = len(dl)
            print(f'Desde {mipais} el número total de registros son:', n)
            print(dl)

        elif opEst == '2': #Devuelve los libros más leidos de forma descendente
            consultaLib = session.execute('select nombre_libro, count(nombre_libro) as "n" from uprofile.plataforma_de_libros group by categoria, nombre_libro;')
            #gurardamos valores en listas para después trabajar en DataFrames
            nombres = []
            count = []
            for cons in consultaLib:
                nombres.append(cons.nombre_libro)
                count.append(cons.n)
            #Creamos el dataframe
            dfFrec = pd.DataFrame({'nombre_libro':nombres,'Frecuencia':count},columns=['nombre_libro','Frecuencia'])
            dfFrec = dfFrec.sort_values(by=['Frecuencia'], ascending=[False]).head(3) #mostramos los 3 más leidos por categoria
            print("Los tres libros más populares son:")
            print(dfFrec)

        elif opEst == '3': #Devuelve las puntuaciones que usuarios en general han hecho sobre los libros
            nomLibro = str(input("Ingresa el nombre del libro: "))
            texto = "'"+nomLibro+"'" #conctenamos texto para poder realizar la consulta.

            consultaH = session.execute('select membresia,nombre_cliente, nombre_libro, puntuacion from uprofile.plataforma_de_libros where nombre_libro ='+ str(texto) +' allow filtering;')
            #guardamos todo en listas para trabajar con data Frames
            membresias = []
            nombres = []
            puntuacion = []
            clientes = []
            for cons in consultaH:
                membresias.append(cons.membresia)
                nombres.append(cons.nombre_libro)
                puntuacion.append(cons.puntuacion)
                clientes.append(cons.nombre_cliente)

            #Creamos dataFrame

            dfLib = pd.DataFrame({'membresia':membresias,'nombre_cliente':clientes,'nombre_libro':nombres,'puntuacion':puntuacion},columns=['membresia','nombre_cliente','nombre_libro','puntuacion'])
            print(dfLib)


        elif opEst == '4':
            break;
        
        else:
            print("Opción no válida. Por favor, vuelva a intentarlo.")

            
        input("Presione ENTER para continuar.")
        print()
            
        
# Iniciamos con la interfaz grafica y todo su desarrollo    
while True:
    # Inicializamos llamando al menu y eligiendo una opción.
    menu_aplication()
    opMenu = input("Ingrese una opción >> ")
    print()
    clear()
               
    if opMenu == '1': #Ingresar un nuevo registro.
        # En este caso, el usuario va a introducir los datos que
        # se le van a agregar a la tabla: nombre_cliente, nombre_libro, 
        # membresia, pais, categoria
        print('-'*5," Ingresar Registro:", '-'*5, '\n')
        memb = int(input("Membresía del cliente: "))
        cliente = str(input("Nombre del cliente: "))
        pais = str(input("País del cliente: "))
        libro = str(input("Nombre del libro: "))
        categ = str(input("Cateogría del libro: "))
        punt = float(input("Puntuación del libro: "))
        
        # insertamos los datos en la tabla
        session.execute(insertar_datos, [memb,cliente,pais,libro,categ,punt])
    
    elif opMenu == '2': # Obtener toda la tabla
        print('-'*5, "Obtener todos los registros de la tabla", '-'*5, '\n')
        consulta = session.execute('select * from uprofile.plataforma_de_libros;')
        membresias = []
        categorias = []
        clientes = []
        paises = []
        libros = []
        puntuaciones = []
        
        for cons in consulta:
            membresias.append(cons.membresia)
            categorias.append(cons.categoria)
            clientes.append(cons.nombre_cliente)
            paises.append(cons.pais)
            libros.append(cons.nombre_libro)
            puntuaciones.append(cons.puntuacion)
            
        #Creamos el data frame
        cons2 = pd.DataFrame({'membresia':membresias,'nombre_cliente':clientes,'pais':paises,'nombre_libro':libros,'categoria':categorias,'puntuacion':puntuaciones},
                            columns=['membresia','nombre_cliente','pais','nombre_libro','categoria','puntuacion'])
        
        pd.set_option('display.max_rows', len(cons2))
        display(cons2)
        pd.reset_option('display.max_rows')
            
            
            
    elif opMenu == '3': # Obtener todos los registros de un usuario
        print('-'*5, "Obtener todos los registros de un cliente", '-'*5, '\n')
        memb = int(input("Ingrese la membresia del cliente: "))
        # Realizamos la consulta
        consulta = session.execute("select membresia,nombre_cliente,pais,nombre_libro,categoria,puntuacion from uprofile.plataforma_de_libros where membresia = "+str(memb)+" allow filtering;")
        for cons in consulta:
            print(cons.membresia, cons.nombre_cliente, cons.pais, cons.nombre_libro, cons.categoria, cons.puntuacion)
    
    elif opMenu == '4': #  Obtener la cantidad de libros leídos de un cliente
        print('-'*5, "Obtener la cantidad de libros leídos de un cliente", '-'*5, '\n')
        memb = int(input("Ingrese la membresia del cliente: "))
        # Realizamos la consulta
        consulta = session.execute("select membresia,nombre_cliente,pais,count(*) as libros_leidos from uprofile.plataforma_de_libros where membresia = "+str(memb)+" allow filtering;")
        for cons in consulta:
            print(cons.membresia, cons.nombre_cliente, cons.pais, cons.libros_leidos)
    
    elif opMenu == '5': #  Obtener la cantidad de libros leídos de un cliente por categoría
        print('-'*5, "Obtener la cantidad de libros leídos de un cliente por categoría", '-'*5, '\n')
        memb = int(input("Ingrese la membresia del cliente: "))
        # Realizamos la consulta
        consulta = session.execute("select membresia,nombre_cliente,pais, categoria, count(*) as libros_leidos_por_categoria from uprofile.plataforma_de_libros where membresia = "+str(memb)+"  allow filtering;")
        for cons in consulta:
            print(cons.membresia, cons.nombre_cliente, cons.pais, cons.categoria, cons.libros_leidos_por_categoria)
        
        
    elif opMenu == '6': #Obtener la categoria preferida de un usuario
        print('-'*5, "Obtener la categoria preferida de un usuario", '-'*5, '\n')
        memb = int(input("Ingrese la membresia del cliente: "))
        # Realizamos la consulta
        consulta = session.execute('select membresia, nombre_cliente, categoria, avg(puntuacion) as "Promedio" from uprofile.plataforma_de_libros where membresia = '+str(memb)+' group by categoria allow filtering;')
        # Dado que en Cassandra no se pueden realizar consultas anidades, la consulta
        # anterior solo arroja los promedios de la categoria, entonces para mostrar
        # la categoria con mayor promedio en la puntuacion, guardamos los promedios
        # de las puntuaciones en una lista vacia y posteriormente mostramos
        # el que tiene el promedio más grande
        maximo = []
        for cons in consulta:
            maximo.append(cons.Promedio)
        consulta = session.execute('select membresia, nombre_cliente, categoria, avg(puntuacion) as "Promedio" from uprofile.plataforma_de_libros where membresia = '+str(memb)+' group by categoria allow filtering;')
        for cons in consulta:
            if cons.Promedio == max(maximo):
                print(cons.membresia, cons.nombre_cliente, cons.categoria, cons.Promedio)            
    
    elif opMenu == '7': #Obtener los clientes que más disfrutaron de un libro
        print('-'*5, 'Obtener los clientes que más disfrutaron de un libro', '-'*5, '\n')
        libro = input('Ingrese el nombre del libro: ')
        # Encerramos el libro entre comillas
        libro = '\''+libro+'\''
        # Hacemos la consulta en Cassandra
        consulta = session.execute("select membresia,nombre_cliente,nombre_libro, puntuacion from uprofile.plataforma_de_libros where nombre_libro = "+libro+" and puntuacion > 8 allow filtering;")
        # Imprimimos la consulta    
        for cons in consulta:
            print(cons.membresia, cons.nombre_libro, cons.puntuacion)
    
    elif opMenu == '8':
        categ = input('Ingresa la categoría que te interesa conocer: ')
        print('-'*5, f'Los mejores libros para la categoría {categ} son: ', '-'*5, '\n')
        consulta4 = session.execute('select nombre_libro, categoria, avg(puntuacion) as "catprom" from uprofile.plataforma_de_libros group by categoria, nombre_libro;')
        
        categoria = []
        nombre = []
        catProm = []
        for cons in consulta4:
            categoria.append(cons.categoria)
            nombre.append(cons.nombre_libro)
            catProm.append(cons.catprom)
            
        #Creamos el dataframe
        df = pd.DataFrame({'categoria':categoria,'nombre_libro':nombre,
                  'catprom':catProm},columns=['categoria','nombre_libro','catprom'])
        
        df2 = df[df['categoria'] == categ]
        df3=df2.sort_values(by=['catprom'],ascending=[False])
        print(df3)
        
    elif opMenu == '9': #Obtener los libros favoritos de un usuario
        print('-'*5, "Obtener los libros favoritos de un usuario", '-'*5, '\n')
        memb = int(input("Ingresa la membresia del cliente: "))
        # Realizamos la consulta
        consulta = session.execute("select membresia, nombre_cliente, nombre_libro from uprofile.plataforma_de_libros where membresia = "+str(memb)+" and puntuacion > 8 allow filtering;")
        for cons in consulta:
            print(cons.membresia, cons.nombre_cliente, cons.nombre_libro)
        
    elif opMenu == '10':
        estadisticas()

    elif opMenu == '11':
            break;
    else:
        print("Opción no válida. Por favor, vuelva a intentarlo.")
    input("\n\n Presione ENTER para continuar.")
    print()
    
    clear()

++++++++++ PLATAFORMA DE LIBROS ++++++++++ 

++++++++++++++++++++  MENU  ++++++++++++++++++++ 

1.- Ingresar nuevo registro
2.- Obtener todos los registros de la tabla
3.- Obtener todos los registros de un cliente
4.- Obtener la cantidad de libros leídos de un cliente
5.- Obtener la cantidad de libros leídos de un cliente por categoría
6.- Obtener la categoría preferida de un usuario
7.- Obtener los clientes que más disfrutaron de un libro
8.- Obtener los mejores libros de una categoría dada
9.- Obtener los libros favoritos de un usuario
10.- Estadísticas generales
11.- Salir

Ingrese una opción >> 2

----- Obtener todos los registros de la tabla ----- 



membresia                nombre_cliente       pais  \
0   111002674        Mauricio José Barreiro      India   
1   315284557       Yoselyn Juárez Casimiro     Italia   
2   673511509          Misael López Sánchez     España   
3   123456789    Alejandro Pimentel Alarcón    Francia   
4   315232905       Yazmín Baltazar Saucedo    Francia   
5   316281810            Angel Dimas Huesca     España   
6   314129420       Patricia Caire Ascencio      Rusia   
7   315293230           Adriana Leal Molina     Italia   
8   417039617         César López Domínguez   Alemania   
9   418003781        Evelyn Salinas Ramírez    Ecuador   
10  315309490        Brandon Hernandez Nava      China   
11  673511509          Misael López Sánchez     España   
12  315083532         Leonardo Flores Ochoa  Argentina   
13  315335130         Marcela Fuentes Cerda        USA   
14  404114851     Minerva Del Real Martínez     España   
15  315284557       Yoselyn Juárez Casimiro     Italia   
16  316032454      Alejandro Flores Delgado      China   
17  419004066        Ibeth Hernández Corona    Uruguay   
18  315137817          Alberto Gómez Correa     Canadá   
19  315335130         Marcela Fuentes Cerda        USA   
20  312284510          Armando Aguilar Léon     México   
21  418004719     José de Jesús Tapia López     México   
22  315137817          Alberto Gómez Correa     Canadá   
23  315138670           María Bolon Sánchez      China   
24  316063443        Susan Morales Gónzalez     España   
25  315083532         Leonardo Flores Ochoa  Argentina   
26  316178352   Sebastián_Lazcano Espindola      India   
27  417042329         Jasson Gónzalez Ramos      Japón   
28  315335130         Marcela Fuentes Cerda        USA   
29  418003712          Adrián Flores Pulido        USA   
30  315018086             Diego López Reyes  Argentina   
31  315026487        Alexis Medina Mauricio  Guatemala   
32  315295306            Antonio Mani Yañez  Guatemala   
33  315316409        Augusto Medina Ramírez     Panama   
34  316063443        Susan Morales Gónzalez     España   
35  673511509          Misael López Sánchez     España   
36  315132159            Raúl Corona Flores     México   
37  315309490        Brandon Hernandez Nava      China   
38  418004719     José de Jesús Tapia López     México   
39  404114851     Minerva Del Real Martínez     España   
40  315026487        Alexis Medina Mauricio  Guatemala   
41  315042416           Joanna Carmona Lara      India   
42  316294364         Valeria Camacho Ávila      Rusia   
43  404114851     Minerva Del Real Martínez     España   
44  123456789    Alejandro Pimentel Alarcón    Francia   
45  315001091  Epifania Avendaño Montesinos     México   
46  315293230           Adriana Leal Molina     Italia   
47  316063443        Susan Morales Gónzalez     España   
48  673511509          Misael López Sánchez     España   
49  314105086           Jazmín Mendoza Vara    Ecuador   
50  315138670           María Bolon Sánchez      China   
51  315232905       Yazmín Baltazar Saucedo    Francia   
52  418004719     José de Jesús Tapia López     México   
53  314319373            Nadine Mejía Cerda     Panama   
54  316117199          Iván Medina Martínez  Guatemala   
55  312284510          Armando Aguilar Léon     México   
56  418004719     José de Jesús Tapia López     México   
57  315045606     Catherine Jiménez Jiménez    Uruguay   
58  315212992   Esmeralda Gónzalez Palacios     Canadá   
59  315212992   Esmeralda Gónzalez Palacios     Canadá   
60  123456789    Alejandro Pimentel Alarcón    Francia   
61  315132032         César Collado Morales      Rusia   
62  315293230           Adriana Leal Molina     Italia   
63  418004719     José de Jesús Tapia López     México   
64  315212992   Esmeralda Gónzalez Palacios     Canadá   
65  404114851     Minerva Del Real Martínez     España   
66  123456789    Alejandro Pimentel Alarcón    Francia   
67  315295306            Antonio Mani Yañez  Guatemala   
68



 Presione ENTER para continuar.

++++++++++ PLATAFORMA DE LIBROS ++++++++++ 

++++++++++++++++++++  MENU  ++++++++++++++++++++ 

1.- Ingresar nuevo registro
2.- Obtener todos los registros de la tabla
3.- Obtener todos los registros de un cliente
4.- Obtener la cantidad de libros leídos de un cliente
5.- Obtener la cantidad de libros leídos de un cliente por categoría
6.- Obtener la categoría preferida de un usuario
7.- Obtener los clientes que más disfrutaron de un libro
8.- Obtener los mejores libros de una categoría dada
9.- Obtener los libros favoritos de un usuario
10.- Estadísticas generales
11.- Salir

Ingrese una opción >> 10

----- Estadísticas generales ----- 


1.- Registros totales por país
2.- Libros más leidos 
3.- Puntuaciones de usuarios sobre libros en especifico 
4.- Regresar

Ingrese una opción >> 1

Introduce el pais: México
Desde México el número total de registros son: 10
   usuarios  membresia                nombre_cliente
20   México  312284510          Armand

# Conclusiones

<p style='text-align: justify;'>Al trabajar con Cassandra pudimos encontrar una base de datos potente en el momento de realizar lectura de datos, y principalmente resaltamos su principal cualidad, la cual es la $\textbf{escalabilidad}$. La base de datos que trabajamos era pequeña, con menos de 100 registros pero aún así eran los suficientes para poder trabajar y realizar consultas interesantes.

Cassandra se presenta como una buena solución a la problématica de los grandes bancos de datos, aún cuando su manera de modelar datos dista del modelo clásico relacional este posee herramientas intereesantes para administrar **Primary Keys** pues la primera gran diferencia es que en casandra la información se administra por: </p>
* Nodos
* Primary Key
* Cluster Key

<p style='text-align: justify;'>Lo cual nos permite clasificar la información de forma muy diferente a lo que se suele hacer con las bases de datos relacionales, pues los registros se van registrando por renglón, identificados por la $\textbf{primary key}$ y ordenados por la $\textbf{cluster key}$. Una de las ventajas que tiene trabajar con Apache Cassandra es que podemos trabajar con más de un campo como $\textbf{primary Key}$, lo que dependerá del diseño de la base de datos. </p>
    
<p style='text-align: justify;'>La principal ventaja de Apache Cassandra es **la escalabilidad** pues podemos agregar al esquema de nuestra base de datos nuevas columnas sin la necesidad de redistribuir toda la estructura de nuestra Base de Datos, como pasaba con redis donde para poder agregar un nuevo nodo, teniamos que redistribuir toda la estructura del anillo.</p>

<p style='text-align: justify;'>Otra de las diferencias es que Apache Cassandra tiene su propio lenguaje de consulta de datos la cual es **CQL**, el cual nos provee de funciones bastante parecidas a las acostumbradas en **SQL**, con ligeras diferencias como por ejemplo que en CQL no existe el ORDER BY. Además, para poder usar la función GROOUP BY, necesariamente se tiene que realizar con la $\textit{partition key}$, y no se puede con ningún otro campo, estas pueden ser ventajas o desventajas dependiendo del tipo de información que el usuario pretenda almacenar en la base de datos. En nuestro caso, como nosotros habíamos diseñado inicialmente, no habíamos considerado este punto; y como en una de las consultas que nos pedía la práctica teníamos que agrupar por categorías, es por eso que consideramos la columna $\textit{categoria}$ como $\textit{partition key}$; y para que la primary key cumpliera con ser única, seleccionamos los atributos $\textit{nombre_libro, membresia}$ como la $\textit{cluster key}$.</p>

<p style='text-align: justify;'>Como habíamos mencionado antes, dado que no se pueden realizar consultas anidadas, en el desarrollo del menú de la práctica nos hemos apoyado de las herramientas de Python para satisfacer los requerimientos de los objetivos.</p>

<p style='text-align: justify;'>Cassandra es una solución brillante para muchos casos de uso que podemos encontrar en el mundo del Big Data. Sin embargo, no es adecuada para alojar **data warehouse**.</p>

<p style='text-align: justify;'>Lo ideal antes de trabajar con esta base de datos es tener claro desde un principio el caso de uso y el tipo de consultas que realizaremos para diseñar de una manera coherente la base de datos, para poder manejar un gran volúmen de datos aprovechando las cualidades de esta base de datos distribuida.</p>

<p style='text-align: justify;'>Por sus características, Apache Cassandra es una de las Bases de datos de tipo columnar más famosas que hay, algunas de las aplicaciones que la usan son las siguientes: </p>

<img src="cassandra.png" width="300" height="300">

Finalmente, podemos mencionar entonces algunas ventajas y desventajas:

Ventajas: 

* <p style='text-align: justify;'>Cassandra resuelve el problema de los sistemas distribuidos y escalables, y está diseñado para hacer frente a los desafíos de gestión de datos de las empresas modernas.</p>

* <p style='text-align: justify;'>Cassandra es un sistema descentralizado: no existe un único punto de falla, si está presente la configuración mínima requerida para el clúster: cada nodo del clúster tiene la misma función y cada nodo puede atender cualquier solicitud. Las estrategias de replicación se pueden configurar. Es posible agregar nuevos nodos al clúster de servidores de manera muy fácil. Además, si un nodo falla, los datos se pueden recuperar de algunos de los otros nodos (se puede ajustar la redundancia). Es especialmente adecuado para la implementación de múltiples centros de datos, redundancia, recuperación de fallas y recuperación ante desastres, con posibilidad de replicación en múltiples centros de datos.</p>

* Cassandra tiene integración Hadoop, con soporte MapReduce, también para Apache Pig y Apache Hive.

Desventajas:

* No hay integridad referencial, no hay un concepto de conexiones JOIN en Cassandra.
* Las opciones de consulta para recuperar datos son muy limitadas.
* <p style='text-align: justify;'>La clasificación de datos es una decisión de diseño; se puede hacer a través de una de las formas predefinidas; los datos pueden recuperarse en el mismo orden; eso es todo: no hay cosas como ORDER BY ni subconsultas.</p>
* <p style='text-align: justify;'>Diseño de base de datos diferente; en RDBMS pensamos primero en el modelado de datos y luego creamos consultas; aquí, pensamos primero en las consultas más comunes, y después de eso, los datos se modelan en torno a esas consultas.</p>

<p style='text-align: justify;'>Los modelos de base de datos NoSQL no pueden reemplazar completamente la tecnología RDBMS, pero la importancia de NoSQL crecerá debido a la escala, flexibilidad y facilidad de uso. Estamos tratando con más y más datos; queremos aplicaciones duraderas y tolerantes a fallas; queremos aplicaciones que escalen y aplicaciones que sean rápidas. Debido a todo esto, NoSQL nos rodeará cada vez más y definitivamente vale la pena explorar la tecnología.</p>